In [1]:
from datasets import load_dataset
import pandas as pd
from datasets import Dataset, DatasetDict
from dotenv import dotenv_values
from utils.process_split import Process_split 


HF_TOKEN = dotenv_values(".env.base")['HF_TOKEN']
#DATASET_CHEKPOINT = dotenv_values(".env.base")['DATASET_CHEKPOINT']
hf_e3c = load_dataset("bio-datasets/e3c")

/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/.env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
processed_split = Process_split.apply(hf_e3c, 'it.layer1', enitites_separator_in_output="|||")
processed_split['text'] = processed_split['input'] + processed_split['output']
data_ft = Dataset.from_pandas(processed_split)

DatasetDict({
    en.layer1: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 84
    })
    en.layer2: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 171
    })
    en.layer2.validation: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 19
    })
    en.layer3: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 9779
    })
    es.layer1: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 81
    })
    es.layer2: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 162
    })
    es.layer2.validation: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 18
    })
    es.layer3: Dataset({
        features: ['input', 'output', 'language', 'layer', 'text'],
        num_rows: 18

In [2]:
data_ft = pd.DataFrame(columns=['input', 'output', 'layer', 'text'])
splits_dict = {}
splits = ['en.layer1', 'en.layer2', 'en.layer2.validation', 'en.layer3',
          'es.layer1', 'es.layer2', 'es.layer2.validation', 'es.layer3',
          'eu.layer1', 'eu.layer2', 'eu.layer2.validation', 'eu.layer3',
          'it.layer1', 'it.layer2', 'it.layer2.validation', 'it.layer3',
          'fr.layer1', 'fr.layer2', 'fr.layer2.validation', 'fr.layer3']

for split_name in splits:
    processed_split = Process_split.apply(hf_e3c, split_name, enitites_separator_in_output="|||")
    processed_split['text'] = processed_split['input'] + processed_split['output']
    data_ft = Dataset.from_pandas(processed_split)
    splits_dict[split_name] = data_ft
    

In [9]:
ddict = DatasetDict(splits_dict)
ddict.push_to_hub("ferrazzipietro/e3c_finetuning_processed", token=HF_TOKEN)

Uploading the dataset shards: 100%|██████████| 2/2 [01:49<00:00, 54.89s/it]
/Users/pietroferrazzi/Desktop/dottorato/mistral_finetuning/.env/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'list_files_info' (from 'huggingface_hub.hf_api') is deprecated and will be removed from version '0.23'. Use `list_repo_tree` and `get_paths_info` instead.
  warnings.warn(warning_message, FutureWarning)


# Diagnostic

In [16]:
# DA TENERE SOTTO CONTROLLO LA LUNGHEZZA MAX DI CONTESTO....
longest_text = max(splits_dict['it.layer1']['text'], key=len)
print(len(longest_text)*0.75)

8400.75


In [8]:
splits_dict['it.layer1']['output'][0]

'offset: [47, 55] text: ematuria ||| offset: [88, 94] text: trauma ||| offset: [131, 136] text: massa ||| offset: [161, 169] text: partenza ||| offset: [210, 215] text: massa ||| offset: [250, 262] text: secondarismi ||| offset: [293, 306] text: ureterectomia ||| offset: [311, 318] text: referto ||| offset: [332, 338] text: tumore ||| offset: [393, 402] text: anaplasia ||| offset: [425, 431] text: inizia ||| offset: [432, 445] text: chemioterapia ||| offset: [454, 464] text: protocollo ||| offset: [506, 510] text: stop ||| offset: [511, 518] text: terapia ||| offset: [520, 522] text: TC ||| offset: [571, 579] text: immagini ||| offset: [644, 651] text: Biopsia ||| offset: [686, 695] text: Metastasi ||| offset: [804, 813] text: anaplasia ||| offset: [830, 836] text: inizia ||| offset: [837, 844] text: terapia ||| offset: [865, 874] text: raccoglie ||| offset: [892, 905] text: chemioterapia ||| offset: [923, 935] text: carboplatino ||| offset: [947, 956] text: etoposide ||| offset: [969,

In [10]:
longest_string = max(splits_dict['it.layer1']['output'], key=len)
print(len(longest_string)*0.7)


5348.7


In [11]:
longest_string = max(splits_dict['it.layer1']['input'], key=len)
print(len(longest_string)*0.7)


2879.7999999999997


In [ ]:
data_ft = Dataset.from_pandas(data_ft)
data_ft.push_to_hub(DATASET_CHEKPOINT, token=HF_TOKEN)